In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import timeit
#!pip install torchattacks
import torchattacks
import torch
from torchvision import io

In [ ]:
#!pip install git+https://github.com/RobustBench/robustbench.git
import robustbench
from robustbench.utils import load_model,clean_accuracy
from robustbench.data import load_cifar10

# Load models and data

In [ ]:
import os
#os.chdir('D:\\PFE black\\3.Robustbench and torchattacks')
os.getcwd()

In [ ]:
from cifar10_models.resnet import resnet50
my_model = resnet50(pretrained=True)
my_model.eval() 

In [ ]:
images_png = [name for name in os.listdir("D:\\PFE black\\4. Skip gradient method cifar10\\skip-connections-matter\\adv_images_cifar10") if os.path.splitext(name)[-1] == '.png']
images_png = [int(v.split(".")[0][1:]) for v in images_png]
images_png.sort()
images_png = ["n"+str(v)+".png.png"for v in images_png]

In [ ]:
n_images = len(images_png)
adv_images_SGM = torch.zeros(n_images, 3, 32, 32, dtype=torch.uint8)

for i, image_png in enumerate(images_png):
    adv_images_SGM[i] = io.read_image(os.path.join("D:\\PFE black\\4. Skip gradient method cifar10\\skip-connections-matter\\adv_images_cifar10", image_png))
    
#make sure to change type of tensor to float and normalize between 0 and 1
adv_images_SGM = adv_images_SGM.float() / 255

In [ ]:
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.Normalize(mean = [0.4914, 0.4822, 0.4465], std = [0.2471, 0.2435, 0.2616])
])
adv_images_SGM = preprocess(adv_images_SGM)
plt.imshow(adv_images_SGM[0,:,:,:].permute(1, 2, 0) )

In [ ]:
dict_y_test = np.load("cifar10_image_to_label.npy", allow_pickle = True)[()]
y_test = torch.Tensor(list(dict_y_test.values()))
y_test = y_test.type(torch.LongTensor)
y_test

In [ ]:
model_rank1 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf')

In [ ]:
model_rank2 = load_model(model_name='Gowal2021Improving_70_16_ddpm_100m', dataset='cifar10', threat_model='Linf')

In [ ]:
model_rank3 = load_model(model_name='Gowal2020Uncovering_70_16_extra', dataset='cifar10', threat_model='Linf')

In [ ]:
model_rank4 = load_model(model_name='Rebuffi2021Fixing_106_16_cutmix_ddpm', dataset='cifar10', threat_model='Linf')

In [ ]:
model_rank5 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_ddpm', dataset='cifar10', threat_model='Linf')

In [ ]:
model_vanilla = load_model(model_name='Standard', dataset='cifar10', threat_model='Linf')

# torchattacks

### SGM

In [ ]:
SGM_results = []

In [ ]:
acc = clean_accuracy(model_rank1, adv_images_SGM, y_test)
print('Robust Acc rank1: %2.2f %%'%(acc*100))
SGM_results.append(acc)

In [ ]:
acc = clean_accuracy(model_rank2, adv_images_SGM, y_test)
print('Robust Acc rank2: %2.2f %%'%(acc*100))
SGM_results.append(acc)

In [ ]:
acc = clean_accuracy(model_rank3, adv_images_SGM, y_test)
print('Robust Acc rank3: %2.2f %%'%(acc*100))
SGM_results.append(acc)

In [ ]:
acc = clean_accuracy(model_rank4, adv_images_SGM, y_test)
print('Robust Acc rank4: %2.2f %%'%(acc*100))
SGM_results.append(acc)

In [ ]:
acc = clean_accuracy(model_rank5, adv_images_SGM, y_test)
print('Robust Acc rank5: %2.2f %%'%(acc*100))
SGM_results.append(acc)

In [ ]:
acc = clean_accuracy(model_vanilla, adv_images_SGM, y_test)
print('Robust Acc vanilla: %2.2f %%'%(acc*100))
SGM_results.append(acc)

In [ ]:
SGM_results